**Cell-1: Imports & Global Config**

In [1]:
import os
import json
import time
import random
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models
from PIL import Image

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


**Cell-2: Reproducibility (Mandatory)**

In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

**Cell-3: Dataset Paths**

In [3]:
DATASET_DIR = "/kaggle/input/fishes/Fishes"
BATCH_SIZE = 32
IMG_SIZE = 160
EPOCHS = 100
TEMPERATURE = 0.5


**Cell-4: SSL Augmentation (SimCLR style)**

In [4]:
ssl_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4,0.4,0.4,0.1),
    transforms.RandomGrayscale(p=0.2),
    transforms.GaussianBlur(kernel_size=3),
    transforms.ToTensor()
])


**Cell-5: SimCLR Dataset (Two Views)**

In [5]:
class SimCLRDataset(Dataset):
    def __init__(self, root, transform):
        self.samples = []
        for cls in os.listdir(root):
            cls_path = os.path.join(root, cls)
            if os.path.isdir(cls_path):
                for img in os.listdir(cls_path):
                    if img.lower().endswith((".jpg",".png",".jpeg")):
                        self.samples.append(os.path.join(cls_path, img))
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img = Image.open(self.samples[idx]).convert("RGB")
        return self.transform(img), self.transform(img)


**Cell-6: DataLoader**

In [6]:
ssl_dataset = SimCLRDataset(DATASET_DIR, ssl_transform)
ssl_loader = DataLoader(
    ssl_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

len(ssl_dataset)


583

**Cell-7: SimCLR Model (Encoder + Projection Head)**

In [7]:
class SimCLR(nn.Module):
    def __init__(self, base_model="resnet50", proj_dim=128):
        super().__init__()

        self.encoder = models.resnet50(weights=None)
        self.encoder.fc = nn.Identity()   # output: [B, 2048]

        self.projector = nn.Sequential(
            nn.Linear(2048, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, proj_dim)
        )

    def forward(self, x):
        h = self.encoder(x)          # [B, 2048]
        z = self.projector(h)        # [B, proj_dim]
        return F.normalize(z, dim=1)


**Cell-8: NT-Xent Loss (Contrastive Loss)**

In [8]:
def nt_xent_loss(z1, z2, temperature):
    N = z1.size(0)
    z = torch.cat([z1, z2], dim=0)
    sim = F.cosine_similarity(z.unsqueeze(1), z.unsqueeze(0), dim=2)

    sim /= temperature
    labels = torch.arange(N).to(device)
    labels = torch.cat([labels + N, labels])

    mask = torch.eye(2*N, dtype=torch.bool).to(device)
    sim.masked_fill_(mask, -9e15)

    return F.cross_entropy(sim, labels)


**Cell-9: Initialize Model & Optimizer**

In [9]:
model = SimCLR().to(device)
optimizer = optim.Adam(model.parameters(), lr=3e-4)


**Cell-10: Checkpoint Utils (Mandatory)**

In [10]:
CKPT_PATH = "/kaggle/working/simclr_ckpt.pt"

def save_ckpt(epoch, best_loss):
    torch.save({
        "epoch": epoch,
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "best_loss": best_loss
    }, CKPT_PATH)

def load_ckpt():
    if os.path.exists(CKPT_PATH):
        ckpt = torch.load(CKPT_PATH)
        model.load_state_dict(ckpt["model"])
        optimizer.load_state_dict(ckpt["optimizer"])
        return ckpt["epoch"], ckpt["best_loss"]
    return 0, float("inf")


**Cell-11: SimCLR Pretraining Loop (Label-Free)**

In [11]:
start_epoch, best_loss = load_ckpt()
print("Resuming from epoch:", start_epoch)

scaler = torch.amp.GradScaler("cuda")

for epoch in range(start_epoch, EPOCHS):
    model.train()
    epoch_loss = 0.0

    torch.cuda.empty_cache()  # once per epoch

    for x1, x2 in tqdm(ssl_loader):
        x1 = x1.to(device, non_blocking=True)
        x2 = x2.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with torch.amp.autocast("cuda"):
            z1 = model(x1)
            z2 = model(x2)
            loss = nt_xent_loss(z1, z2, TEMPERATURE)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()

    epoch_loss /= len(ssl_loader)
    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {epoch_loss:.4f}")

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        save_ckpt(epoch+1, best_loss)

torch.cuda.empty_cache()


Resuming from epoch: 0


100%|██████████| 18/18 [01:14<00:00,  4.15s/it]


Epoch [1/100] Loss: 4.1305


100%|██████████| 18/18 [01:01<00:00,  3.41s/it]


Epoch [2/100] Loss: 3.9944


...

Epoch [98/100] Loss: 2.8876


100%|██████████| 18/18 [01:03<00:00,  3.51s/it]


Epoch [99/100] Loss: 2.8900


100%|██████████| 18/18 [01:04<00:00,  3.59s/it]

Epoch [100/100] Loss: 2.8744


**Cell-12: Save Frozen Encoder (Deliverable)**

In [12]:
torch.save(model.encoder.state_dict(), "encoder_simclr.pt")
print("SimCLR encoder saved")


SimCLR encoder saved


**Cell-13: Feature Extraction Dataset**

In [13]:
class FeatureDataset(Dataset):
    def __init__(self, root, transform):
        self.samples = []
        self.labels = []
        self.classes = sorted(os.listdir(root))
        cls2idx = {c:i for i,c in enumerate(self.classes)}

        for c in self.classes:
            for img in os.listdir(os.path.join(root,c)):
                if img.lower().endswith((".jpg",".png",".jpeg")):
                    self.samples.append(os.path.join(root,c,img))
                    self.labels.append(cls2idx[c])

        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img = Image.open(self.samples[idx]).convert("RGB")
        return self.transform(img), self.labels[idx]


**Cell-14: Linear Probe Transform**

In [14]:
eval_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])


**Cell-15: Extract Features**

In [15]:
model.encoder.eval()

features, labels = [], []

dataset = FeatureDataset(DATASET_DIR, eval_tf)
loader = DataLoader(dataset, batch_size=64, shuffle=False)

with torch.no_grad():
    for x, y in tqdm(loader):
        x = x.to(device)
        h = model.encoder(x)
        features.append(h.cpu())
        labels.append(y)

X = torch.cat(features).numpy()
y = torch.cat(labels).numpy()


100%|██████████| 10/10 [00:53<00:00,  5.35s/it]


**Cell-16: k-NN Evaluation (Guideline Required)**

In [16]:
for k in [1,5,20]:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X, y)
    pred = knn.predict(X)
    acc = accuracy_score(y, pred)
    print(f"k-NN (k={k}) accuracy: {acc:.4f}")


k-NN (k=1) accuracy: 0.9949
k-NN (k=5) accuracy: 0.8113
k-NN (k=20) accuracy: 0.6278
